**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Obtenga las letras asociadas a cada clave para el siguiente archivo.

In [ ]:
%%writefile input.txt
0 	 C,F,A,B,D,I,H
1 	 A,H,C,I,F,D,B
2 	 B,H,I,F,G
3 	 C,B,D
4 	 D,C,I,G,H
5 	 B,D,C,H,A
6 	 H,D,C,B,G,F,D
7 	 F,A,G,C,B,D,H,I
8 	 F,A,I,B,D
9 	 F,A,B,D,C,D,G,I
10 	 D,B,A,C,H
11 	 G,D,B,H,I,C,F
12 	 D,D,C,F,B,A,H,G
13 	 F,A,D
14 	 D,A,C,G
15 	 H,A,F,D,B,C,G,I
16 	 A,I,B,D
17 	 C,B,G,A,D,H,F
18 	 I,B,A,H,D,F
19 	 B,D,F,D,I
20 	 C,B,H,F,I,G,D,D
21 	 F,A,B,C,G,D
22 	 I,G,F,C,H,B
23 	 H,F,C,B,D,D,A
24 	 F,D,G,A,H
25 	 G,H,B,C,A,F,I
26 	 G,F,B,A,H,D,D,I
27 	 B,A,H,I,D,G,F
28 	 A,H,D,F,C
29 	 C,D,A,F,G,B,H,D

## Mapper

In [ ]:
%%writefile mapper.py
#! /usr/bin/env python

##
## Esta es la función que mapea la entrada a parejas (clave, valor)
##
import sys
import re


##
## Se usa una clase iterable para implementar el mapper.
##

class Mapper:
    
    def __init__(self, stream):
        ## 
        ## almacena el flujo de entrada como una
        ## variable del objeto
        ##
        self.stream = stream
    
    def emit(self, key, value):
        ##
        ## escribe al flujo estándar de salida
        ##
        sys.stdout.write("{}\t{}\n".format(key, value))
            
    def status(self, message):
        ##
        ## imprime un reporte en el flujo de error
        ## no se debe usar el stdout, ya que en este 
        ## unicamente deben ir las parejas (key, value)
        ##
        sys.stderr.write('reporter:status:{}\n'.format(message))
        
    def map(self):

        word_counter = 0
        
        for word in self:
            
            ##
            ## imprime un mensaje indicando la palabra procesada
            ##
            #self.status('procesando ' + word[1])
            
            ##
            ## cuenta la cantidad de palabras procesadas
            ##
            ##
            ## por cada palabra del flujo de datos
            ## emite la pareja (word, 1)
            ##
            self.emit(key=word[0], value=word[1])
            
    def __iter__(self):
        ##
        ## itera sobre cada linea de código recibida
        ## a través del flujo de entrada
        ##
        for line in self.stream:
            ##
            ## itera sobre cada palabra de la línea
            ## (en los ciclos for, retorna las palabras
            ## una a una)
            ##
            #yield line[0]
            #yield line
            groups = re.search("([0-9][0-9]?) .* ([A-Z],.*)$",line)
            #groups = re.search("(.{2}\..{1})$",line)
            key = groups.group(1)
            #letra = line[0]
            value = groups.group(2)
            #valor2 = groups.group(2)
            vector =  [key,value]
            #print(vector)
            yield vector
            #for word in line.split():
                
                #fecha = word[1]
                #yield fecha
                #for content in fecha.split('-'):
                    #mes=content[1]
                    #yield mes
                ##
                ## retorna la palabra siguiente en el
                ## ciclo for
                ##
                #yield word
    

if __name__ == "__main__": 
    ##
    ## inicializa el objeto con el flujo de entrada
    ##
    mapper = Mapper(sys.stdin)
    
    ##
    ## ejecuta el mapper
    ##
    mapper.map()

## Reducer

In [ ]:
%%writefile reducer.py
#!/usr/bin/env python

import sys
import itertools
from operator import itemgetter

class Reducer:
    
    def __init__(self, stream):
        self.stream = stream
        
    def emit(self, key, value):
        sys.stdout.write("{}\t{}\n".format(key, value)) 

    def reduce(self):
        ##
        ## Esta función reduce los elementos que 
        ## tienen la misma clave
        ##
        for key, group in itertools.groupby(sorted(self,key=itemgetter(0),reverse=False), lambda x: x):
        
            for _, val in group:
                val=val.replace(',','')
                val=val.replace('\n','')
                valord=''.join(sorted(val,reverse=False))
                txt=','.join(valord)
                self.emit(key=_, value=txt)
            
            

    def __iter__(self):
        
        for line in self.stream:
            ##
            ## Lee el stream de datos y lo parte 
            ## en (clave, valor)
            ##
            key, val = line.split("\t") 
            #array = [float(val[0]),float(val[1])]
            #out = int(val)
            ##
            ## retorna la tupla (clave, valor)
            ## como el siguiente elemento del ciclo for
            ##
            yield (int(key), val)


if __name__ == '__main__': 
  
    reducer = Reducer(sys.stdin)
    reducer.reduce()

## Ejecución

In [ ]:
%%bash
rm -rf output
STREAM=$HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar
chmod +x mapper.py
chmod +x reducer.py
hadoop jar $STREAM -input input.txt -output output  -mapper mapper.py -reducer reducer.py
cat output/part-00000

In [ ]:
!rm -rf mapper.py reducer.py output input.txt

---

Para realizar la evaluación automática de este libro:

* Abra un Terminal.
* Asegurece que esat en la misma carpeta que contiene este notebook.
* Salve el notebook.
* Ejecute el siguiente comando:

      ./gradetool 11-Taller.ipynb

---